## Bài toán
Dữ liệu gồm n văn bản phân vào 10 chủ đề khác nhau. Cần biễu diễn mỗi văn bản dưới dạng một vector số thể hiện cho nội dụng của văn bản đó.

## Mục lục
- Load dữ liệu từ thư mục
- Loại bỏ các stop words
- Sử dụng thư viện để mã hóa TF-IDF cho mỗi văn bản

## Phương pháp mã hóa: TF-IDF
Cho tập gồm $n$ văn bản: $D = \{d_1, d_2, ... d_n\}$. Tập từ điển tương ứng được xây dựng từ $n$ văn bản này có độ dài là $m$
- Xét văn bản $d$ có $|d|$ từ và $t$ là một từ trong $d$. Mã hóa tf-idf của $t$ trong văn bản $d$ được biểu diễn:
\begin{equation}
    \begin{split}
        \text{tf}_{t, d} &= \frac{f_t}{|d|} \\
        \text{idf}_{t, d} &= \log\frac{n}{n_t}, \ \ \ \ n_t = |\{d\in D: t\in d\}| \\
        \text{tf-idf}_{t d} &= \text{tf}_{t, d} \times \text{idf}_{t, d}
    \end{split}
\end{equation}

- Khi đó văn bản $d$ được mã hóa là một vector $m$ chiều. Các từ xuất hiện trong d sẽ được thay bằng giá trị tf-idf tương ứng. Các từ không xuất hiện trong $d$ thì thay là 0

In [2]:
import os
import matplotlib.pyplot as plt
import numpy as np

from sklearn.datasets import load_files
from pyvi import ViTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

%matplotlib inline

## Load dữ liệu từ thư mục

Cấu trúc thư mục như sau 

- data/news_vnexpress/

    - Kinh tế: 
        - bài báo 1.txt 
        - bài báo 2.txt 
    - Pháp luật
        - bài báo 3.txt 
        - bài báo 4.txt 

In [3]:
INPUT = 'data/news_vnexpress'
os.makedirs("images",exist_ok=True)  # thư mục lưu các các hình ảnh trong quá trình huấn luyện và đánh gía

In [4]:
# statistics
print('Các nhãn và số văn bản tương ứng trong dữ liệu')
print('----------------------------------------------')
n = 0
for label in os.listdir(INPUT):
    print(f'{label}: {len(os.listdir(os.path.join(INPUT, label)))}')
    n += len(os.listdir(os.path.join(INPUT, label)))

print('-------------------------')
print(f"Tổng số văn bản: {n}")

Các nhãn và số văn bản tương ứng trong dữ liệu
----------------------------------------------
doi-song: 120
du-lich: 54
giai-tri: 201
giao-duc: 105
khoa-hoc: 144
kinh-doanh: 262
phap-luat: 59
suc-khoe: 162
the-thao: 173
thoi-su: 59
-------------------------
Tổng số văn bản: 1339


In [5]:
# load data
data_train = load_files(container_path=INPUT, encoding="utf-8")
print('mapping:')
for i in range(len(data_train.target_names)):
    print(f'{data_train.target_names[i]} - {i}')

print('--------------------------')
print(data_train.filenames[0:1])
# print(data_train.data[0:1])
print(data_train.target[0:1])
print(data_train.data[0:1])

print("\nTổng số  văn bản: {}" .format( len(data_train.filenames)))

mapping:
doi-song - 0
du-lich - 1
giai-tri - 2
giao-duc - 3
khoa-hoc - 4
kinh-doanh - 5
phap-luat - 6
suc-khoe - 7
the-thao - 8
thoi-su - 9
--------------------------
['data/news_vnexpress\\khoa-hoc\\00133.txt']
[4]
['Mời độc giả đặt câu hỏi tại đây\n']

Tổng số  văn bản: 1339


## Chuyển dữ liệu dạng text về ma trận (n x m) bằng TF-IDF

In [6]:
# load dữ liệu các stopwords 
with open("data/vietnamese-stopwords.txt", encoding="utf-8", errors="ignore") as f:
    stopwords = f.readlines()
stopwords = [x.strip().replace(" ", "_") for x in stopwords] 
print(f"Số lượng stopwords: {len(stopwords)}")
print(stopwords[:10])

# Chuyển hoá dữ liệu text về dạng vector TF 
#     - loại bỏ từ dừng (stop words)
#     - sinh từ điển
module_count_vector = CountVectorizer(stop_words=stopwords)
model_rf_preprocess = Pipeline([('vect', module_count_vector),
                    ('tfidf', TfidfTransformer()),
                    ])
# Hàm thực hiện chuyển đổi dữ liệu text thành dữ liệu số dạng ma trận 
# Input: Dữ liệu 2 chiều dạng numpy.array, mảng nhãn id dạng numpy.array 
data_preprocessed = model_rf_preprocess.fit_transform(data_train.data, data_train.target)

print(f"\nSố lượng từ trong từ điển: {len(module_count_vector.vocabulary_)}")
print(f"Kích thước dữ liệu sau khi xử lý: {data_preprocessed.shape}")
print(f"Kích thước nhãn tương ứng: {data_train.target.shape}")

Số lượng stopwords: 2063
['a_lô', 'a_ha', 'ai', 'ai_ai', 'ai_nấy', 'ai_đó', 'alô', 'amen', 'anh', 'anh_ấy']

Số lượng từ trong từ điển: 12794
Kích thước dữ liệu sau khi xử lý: (1339, 12794)
Kích thước nhãn tương ứng: (1339,)


In [7]:
X = data_preprocessed
Y = data_train.target

In [8]:
X.shape, Y.shape

((1339, 12794), (1339,))

In [23]:
print(X[100].toarray())
print(Y[100])

[[0.         0.         0.         ... 0.         0.14047909 0.        ]]
5


In [24]:
sum(sum(X[100].toarray() != 0))

np.int64(289)

In [25]:
print(X[100])

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 289 stored elements and shape (1, 12794)>
  Coords	Values
  (0, 81)	0.015210600075503207
  (0, 100)	0.020768188378930425
  (0, 156)	0.023261351455612565
  (0, 188)	0.047221897003560664
  (0, 269)	0.03323819172801037
  (0, 392)	0.024782219384691568
  (0, 397)	0.034230713959923546
  (0, 418)	0.04891486528569595
  (0, 662)	0.022768439273483886
  (0, 909)	0.033180077166638476
  (0, 1194)	0.0511669633301605
  (0, 1209)	0.102333926660321
  (0, 1219)	0.0511669633301605
  (0, 1271)	0.01602427109747773
  (0, 1590)	0.034972934749423014
  (0, 1631)	0.023492412503023054
  (0, 1783)	0.04734199730706182
  (0, 1866)	0.0511669633301605
  (0, 2076)	0.014562584276561891
  (0, 2101)	0.026934961949943485
  (0, 2111)	0.025773947875790803
  (0, 2135)	0.04321768742873188
  (0, 2140)	0.01567958623035775
  (0, 2159)	0.01610329399005839
  (0, 2170)	0.02950646667230756
  :	:
  (0, 12270)	0.021258146440605235
  (0, 12452)	0.07589473356362791
  (0, 1250